In [60]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy as sp
strength = 1.0
eps = 1.0
#CCD for pairing model without considering a pair as a particle\


def paired_states(bra):
    x = []
    for i in np.arange(0,len(bra),2):
        x.append((bra[i]%2 != 0 and bra[i+1] == bra[i] + 1))
    res = x[0]
    for i in range(len(x)):
        res = res and x[i]
    return res

def ph_config(bra,ket):
    #returns number of ph excitations of a given state
    return len(set(bra) & set(ket))

def sp_energies(i,eps):
    #single particle (sp) energy in energy state i
    #eps:energy spacing between sp states 
    if i ==1 or i==2:
        return 0.0
    elif i%2 !=0 :
        return (i//2)*eps
    else:
        return (i/2 - 1)*eps




# def V2(bra,ket):
#     if paired_states(bra) == True and paired_states(ket) == True:
#         if ph_config(bra,ket) == 0:
#             return -strength/2.0
#         elif ph_config(bra,ket) == 2:
#                 return -A*strength/4.0
#         else:
#             return 0.0
#     else:
#         return 0.0
    
    
def V2(bra,ket):
    if paired_states(bra) == True and paired_states(ket) == True:
        if ph_config(bra,ket) == 0 or ph_config(bra,ket) == 2:
            return -strength/2.0
    else:
        return 0.0
    
    


no_sp_states = 8
A = 4
#p_h = 2 #CCD

nocc = np.arange(A+1,no_sp_states+1,1,dtype = int)  #not occupied states a,b,c...
nocc_len = len(nocc)
print('not occupied states = ', nocc)


occ = np.arange(1,A+1,1,dtype = int)  #occupied states i,j,k,...
print('occupied states = ', occ)


occ_len = len(occ)


#Since we are interested in only CCD, we are looking at states in ph_dict dictionary 
#Definition of all the arrays
t_vec = np.zeros((nocc_len,nocc_len,occ_len,occ_len))

V_pp_hh = np.zeros((nocc_len,nocc_len,occ_len,occ_len))

V_hh_pp = np.zeros((occ_len,occ_len,nocc_len,nocc_len))

V_pp_pp = np.zeros((nocc_len,nocc_len,nocc_len,nocc_len))

V_hh_hh = np.zeros((occ_len,occ_len,occ_len,occ_len))

f_h_h = np.zeros((occ_len,occ_len))

f_p_p = np.zeros((nocc_len, nocc_len))


print('Setting up matrix V_pp_hh')
for a in nocc:
    for b in nocc:
        for i in occ:
            for j in occ:
                if (j>= i) and (b >= a):
                    #print(a,b,i,j)
                    bra = [a,b]
                    ket = [i,j]
                    V_pp_hh[a-A-1][b-A-1][i-1][j-1] = V2(bra,ket)
                    if V_pp_hh[a-A-1][b-A-1][i-1][j-1] != 0.0 : 
                        V_pp_hh[a-A-1][b-A-1][j-1][i-1] = -V2(bra,ket)
                        V_pp_hh[b-A-1][a-A-1][j-1][i-1] = V2(bra,ket)
                        V_pp_hh[b-A-1][a-A-1][i-1][j-1] = -V2(bra,ket)
                if V_pp_hh[a-A-1][b-A-1][i-1][j-1] != 0:
                    print('a=',a,' ','b=',b,' ','i=',i,' ',\
                          'j=',j,' ','ME =',' ',V_pp_hh[a-A-1][b-A-1][i-1][j-1])

print('Setting up matrix V_hh_pp')
for i in occ:
    for j in occ:
        for a in nocc:
            for b in nocc:
                if (j>= i) and (b >= a):
                    #print(a,b,i,j)
                    bra = [i,j]
                    ket = [a,b]
                    V_hh_pp[i-1][j-1][a-A-1][b-A-1] = V2(bra,ket)
                    if V_hh_pp[i-1][j-1][a-A-1][b-A-1] != 0.0 : 
                        V_hh_pp[j-1][i-1][a-A-1][b-A-1] = -V2(bra,ket)
                        V_hh_pp[j-1][i-1][b-A-1][a-A-1] = V2(bra,ket)
                        V_hh_pp[i-1][j-1][b-A-1][a-A-1] = -V2(bra,ket)
                if V_hh_pp[i-1][j-1][a-A-1][b-A-1] != 0:
                    print('i=',i,' ','j=',j,' ','a=',a,' ',\
                          'b=',b,' ','ME =',' ',\
                          V_hh_pp[i-1][j-1][a-A-1][b-A-1])                    
                    
                    
                    
                    
print('Setting up V_hh_hh:')
for i in occ:
    for j in occ:
        for k in occ:
            for l in occ:
                if (j>= i) and (l >= k):
                    bra = [i,j]
                    ket = [k,l]
                    V_hh_hh[i-1][j-1][k-1][l-1] = V2(bra,ket)
                    if V_hh_hh[i-1][j-1][k-1][l-1] != 0.0 : 
                        V_hh_hh[i-1][j-1][l-1][k-1] = -V2(bra,ket)
                        V_hh_hh[j-1][i-1][l-1][k-1] = V2(bra,ket)
                        V_hh_hh[j-1][i-1][k-1][l-1] = -V2(bra,ket)
        
                if V_hh_hh[i-1][j-1][k-1][l-1] != 0:
                    print('i=',i,' ','j=',j,' ','k=',k,' ',\
                          'l=',l,' ','ME =',' ',V_hh_hh[i-1][j-1][k-1][l-1])
                    
print('Setting up V_pp_pp:')
for a in nocc:
    for b in nocc:
        for c in nocc:
            for d in nocc:
                if (b>= a) and (d >= c):
                    bra = [a,b]
                    ket = [c,d]
                    V_pp_pp[a-A-1][b-A-1][c-A-1][d-A-1] = V2(bra,ket)
                    if V_pp_pp[a-A-1][b-A-1][c-A-1][d-A-1] != 0.0 : 
                        V_pp_pp[a-A-1][b-A-1][d-A-1][c-A-1] = -V2(bra,ket)
                        V_pp_pp[b-A-1][a-A-1][d-A-1][c-A-1] = V2(bra,ket)
                        V_pp_pp[b-A-1][a-A-1][c-A-1][d-A-1] = -V2(bra,ket)
        
                if V_pp_pp[a-A-1][b-A-1][c-A-1][d-A-1] != 0:
                    print('a=',a,' ','b=',b,' ','c=',c,' ',\
                          'd=',d,' ','ME =',' ',\
                          V_pp_pp[a-A-1][b-A-1][c-A-1][d-A-1])
                    

                    
                    
print('Setting up f_h_h:')
for i in occ:
    for j in occ:
        #print(i,j)
        if i==j:
            f_h_h[i-1][j-1] = sp_energies(i,eps)
            #print(i,j,sp_energies(i,eps))
            term = 0.0
            for k in occ:
                term+= V_hh_hh[i-1][k-1][j-1][k-1]
                #print(i,k,j,k,term)
            f_h_h[i-1][j-1] += term
        if f_h_h[i-1][j-1] != 0 : 
            print('i=',i,' ','j=',j,' ','f_i_j =',f_h_h[i-1][j-1])
            
            
print('Setting up f_p_p:')
for a in nocc:
    for b in nocc:
        #print(i,j)
        if a==b:
            f_p_p[a-A-1][b-A-1] = sp_energies(a,eps) 
            #print(i,j,sp_energies(i,eps))
        if f_h_h[a-A-1][b-A-1] != 0 : 
            print('a=',a,' ','b=',b,' ','f_a_b =',f_p_p[a-A-1][b-A-1])


    
def E_ref(A):
    Eref = 0.0
    for i in range(A):
        Eref += sp_energies(i+1,eps)
    term = 0.0
    for i in range(A):
        for j in range(A):
            term += V_hh_hh[i][j][i][j]
    Eref += term*0.5
    return Eref

print(E_ref(A))

chi_b_c = np.zeros((nocc_len,nocc_len))
for b in nocc:
    for c in nocc:
        val = 0.0
        for d in nocc:
            for k in occ:
                for l in occ:
                    val += (V_hh_pp[k-1][l-1][c-A-1][d-A-1]\
                            *t_vec[d-A-1][b-A-1][k-1][l-1])
        chi_b_c[b-A-1][c-A-1] =  val 
        
chi_k_j = np.zeros((occ_len,occ_len)) 
for k in occ:
    for j in occ:
        val = 0.0
        for d in nocc:
            for c in nocc:
                for l in occ:
                    val+= (t_vec[c-A-1][d-A-1][l-1][j-1]*\
                          V_hh_pp[k-1][l-1][c-A-1][d-A-1])
        chi_k_j[k-1][j-1] = val
        

        
        
chi_bk_jc = np.zeros((nocc_len,occ_len,occ_len,nocc_len)) 
for b in nocc:
    for k in occ:
        for j in occ:
            for c in nocc:
                val = 0.0
                for d in nocc:
                    for l in occ:
                        val += (t_vec[d-A-1][b-A-1][l-1][j-1]*\
                               V_hh_pp[k-1][l-1][c-A-1][d-A-1])
                chi_bk_jc[b-A-1][k-1][j-1][c-A-1] = val
                
                
chi_ab_cd = np.zeros((nocc_len,nocc_len,nocc_len,nocc_len))
for a in nocc:
    for b in nocc:
        for c in nocc:
            for d in nocc:
                val = 0.0
                for k in occ:
                    for l in occ:
                        val += (t_vec[a-A-1][b-A-1][k-1][l-1]\
                                *V_hh_pp[k-1][l-1][c-A-1][d-A-1])
                chi_ab_cd[a-A-1][b-A-1][c-A-1][d-A-1] = val
                    
    
    
def H_ab_ij(a,b,i,j,chi_b_c,chi_k_j,chi_bk_jc,chi_ab_cd,t_vec) :

    diagram1 = V_pp_hh[a-A-1][b-A-1][i-1][j-1]
    
    
    diagram2 = 0.0
    for k in occ:
        diagram2+= (-f_h_h[k-1][j-1]*t_vec[a-A-1][b-A-1][i-1][k-1] + \
                    f_h_h[k-1][i-1]*t_vec[a-A-1][b-A-1][j-1][k-1])
        
    diagram3 = 0.0
    for c in nocc:
        diagram3+= (f_p_p[b-A-1][c-A-1]*t_vec[a-A-1][c-A-1][i-1][j-1]\
                    - f_p_p[a-A-1][c-A-1]*t_vec[b-A-1][c-A-1][i-1][j-1])
        
    diagram5 = 0.0
    for k in occ:
        for l in occ:
            diagram5 += (V_hh_hh[k-1][l-1][i-1][j-1]*\
                         t_vec[a-A-1][b-A-1][k-1][l-1])
    diagram5 *= 0.5
    
    
    diagram6 = 0.0
    for c in nocc:
        for d in nocc:
            diagram6 += (V_pp_pp[a-A-1][b-A-1][c-A-1][d-A-1]*\
                         t_vec[c-A-1][d-A-1][i-1][j-1])
    diagram6 *= 0.5
    
    
    diagram7 = 0.0
    for c in nocc:
        diagram7 += (t_vec[a-A-1][c-A-1][i-1][j-1]*\
                     chi_b_c[b-A-1][c-A-1]-\
                     t_vec[b-A-1][c-A-1][i-1][j-1]*\
                     chi_b_c[a-A-1][c-A-1])
    diagram7 *= 0.5
    
        
    diagram8 = 0.0
    for k in occ:
        diagram8 += (t_vec[a-A-1][b-A-1][i-1][k-1]*\
                     chi_k_j[k-1][j-1] - \
                     t_vec[a-A-1][b-A-1][j-1][k-1]*chi_k_j[k-1][i-1])
    diagram8 *= 0.5
    
    
    
    diagram9 = 0.0
    for c in nocc:
        for k in occ:
            diagram9 += (t_vec[a-A-1][c-A-1][i-1][k-1]\
                         *chi_bk_jc[b-A-1][k-1][j-1][c-A-1] -\
                         t_vec[b-A-1][c-A-1][i-1][k-1]\
                         *chi_bk_jc[a-A-1][k-1][j-1][c-A-1] -\
                         t_vec[a-A-1][c-A-1][j-1][k-1]\
                         *chi_bk_jc[b-A-1][k-1][i-1][c-A-1] +\
                         t_vec[b-A-1][c-A-1][j-1][k-1]\
                         *chi_bk_jc[a-A-1][k-1][i-1][c-A-1])
    diagram9 *= 0.5
    
    
    
    diagram10 = 0.0
    for c in nocc:
        for d in nocc:
            diagram10 += (t_vec[c-A-1][d-A-1][i-1][j-1]\
                          *chi_ab_cd[a-A-1][d-A-1][c-A-1][d-A-1])
    diagram10 *= 0.25
    
    
    return (diagram1 + diagram2 + diagram3 + diagram5 + diagram6 + \
            diagram7 + diagram8 + diagram9 +diagram10)
        
    
print("Setting up Hmat_ab_ij :")    
Hmat_ab_ij = np.zeros((nocc_len,nocc_len,occ_len,occ_len))

for a in nocc:
    for b in nocc:
        for i in occ:
            for j in occ:
                Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1] \
                = H_ab_ij(a,b,i,j,chi_b_c,chi_k_j,chi_bk_jc,\
                          chi_ab_cd,t_vec)
                if Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1] != 0.0:
                    print('a=', a, ' ', 'b=',b, ' ', 'i=',i,' ',\
                          'j=',j, 'ME=',\
                          ' ',Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1])


not occupied states =  [5 6 7 8]
occupied states =  [1 2 3 4]
Setting up matrix V_pp_hh
a= 5   b= 6   i= 1   j= 2   ME =   -0.5
a= 5   b= 6   i= 2   j= 1   ME =   0.5
a= 5   b= 6   i= 3   j= 4   ME =   -0.5
a= 5   b= 6   i= 4   j= 3   ME =   0.5
a= 6   b= 5   i= 1   j= 2   ME =   0.5
a= 6   b= 5   i= 2   j= 1   ME =   -0.5
a= 6   b= 5   i= 3   j= 4   ME =   0.5
a= 6   b= 5   i= 4   j= 3   ME =   -0.5
a= 7   b= 8   i= 1   j= 2   ME =   -0.5
a= 7   b= 8   i= 2   j= 1   ME =   0.5
a= 7   b= 8   i= 3   j= 4   ME =   -0.5
a= 7   b= 8   i= 4   j= 3   ME =   0.5
a= 8   b= 7   i= 1   j= 2   ME =   0.5
a= 8   b= 7   i= 2   j= 1   ME =   -0.5
a= 8   b= 7   i= 3   j= 4   ME =   0.5
a= 8   b= 7   i= 4   j= 3   ME =   -0.5
Setting up matrix V_hh_pp
i= 1   j= 2   a= 5   b= 6   ME =   -0.5
i= 1   j= 2   a= 6   b= 5   ME =   0.5
i= 1   j= 2   a= 7   b= 8   ME =   -0.5
i= 1   j= 2   a= 8   b= 7   ME =   0.5
i= 2   j= 1   a= 5   b= 6   ME =   0.5
i= 2   j= 1   a= 6   b= 5   ME =   -0.5
i= 2   j= 1   a= 

In [61]:

def H_ab_ij(a,b,i,j,chi_b_c,chi_k_j,chi_bk_jc,chi_ab_cd,t_vec) :

    diagram1 = V_pp_hh[a-A-1][b-A-1][i-1][j-1]
    
    
    diagram2 = 0.0
    for k in occ:
        diagram2+= (-f_h_h[k-1][j-1]*t_vec[a-A-1][b-A-1][i-1][k-1] + \
                    f_h_h[k-1][i-1]*t_vec[a-A-1][b-A-1][j-1][k-1])
        
    diagram3 = 0.0
    for c in nocc:
        diagram3+= (f_p_p[b-A-1][c-A-1]*t_vec[a-A-1][c-A-1][i-1][j-1]\
                    - f_p_p[a-A-1][c-A-1]*t_vec[b-A-1][c-A-1][i-1][j-1])
        
    diagram5 = 0.0
    for k in occ:
        for l in occ:
            diagram5 += (V_hh_hh[k-1][l-1][i-1][j-1]*\
                         t_vec[a-A-1][b-A-1][k-1][l-1])
    diagram5 *= 0.5
    
    
    diagram6 = 0.0
    for c in nocc:
        for d in nocc:
            diagram6 += (V_pp_pp[a-A-1][b-A-1][c-A-1][d-A-1]*\
                         t_vec[c-A-1][d-A-1][i-1][j-1])
    diagram6 *= 0.5
    
    
    diagram7 = 0.0
    for c in nocc:
        diagram7 += (t_vec[a-A-1][c-A-1][i-1][j-1]*\
                     chi_b_c[b-A-1][c-A-1]-\
                     t_vec[b-A-1][c-A-1][i-1][j-1]*\
                     chi_b_c[a-A-1][c-A-1])
    diagram7 *= 0.5
    
        
    diagram8 = 0.0
    for k in occ:
        diagram8 += (t_vec[a-A-1][b-A-1][i-1][k-1]*\
                     chi_k_j[k-1][j-1] - \
                     t_vec[a-A-1][b-A-1][j-1][k-1]*chi_k_j[k-1][i-1])
    diagram8 *= 0.5
    
    
    
    diagram9 = 0.0
    for c in nocc:
        for k in occ:
            diagram9 += (t_vec[a-A-1][c-A-1][i-1][k-1]\
                         *chi_bk_jc[b-A-1][k-1][j-1][c-A-1] -\
                         t_vec[b-A-1][c-A-1][i-1][k-1]\
                         *chi_bk_jc[a-A-1][k-1][j-1][c-A-1] -\
                         t_vec[a-A-1][c-A-1][j-1][k-1]\
                         *chi_bk_jc[b-A-1][k-1][i-1][c-A-1] +\
                         t_vec[b-A-1][c-A-1][j-1][k-1]\
                         *chi_bk_jc[a-A-1][k-1][i-1][c-A-1])
    diagram9 *= 0.5
    
    
    
    diagram10 = 0.0
    for c in nocc:
        for d in nocc:
            diagram10 += (t_vec[c-A-1][d-A-1][i-1][j-1]\
                          *chi_ab_cd[a-A-1][b-A-1][c-A-1][d-A-1])
    diagram10 *= 0.25
    
    
    return (diagram1 + diagram2 + diagram3 + diagram5 + diagram6 + \
            diagram7 + diagram8 + diagram9 +diagram10)

#     return (diagram1 + diagram2 + diagram3 + diagram5 + diagram6)


def ground_E(t_vec):
    val = 0.0
    for i in occ:
        for j in occ:
            for a in nocc:
                for b in nocc:
                    val += V_hh_pp[i-1][j-1][a-A-1][b-A-1]\
                    *t_vec[a-A-1][b-A-1][i-1][j-1]
                    
    val*= 0.25
    return val+ E_ref(A)
    
    

    
iterations = 60
t_vec = np.zeros((nocc_len,nocc_len,occ_len,occ_len))
for a in nocc:
    for b in nocc:
        for i in occ:
            for j in occ:
                t_vec[a-A-1][b-A-1][i-1][j-1]= (V_pp_hh[a-A-1][b-A-1][i-1][j-1]\
                                               /(f_p_p[a-A-1][a-A-1] +f_p_p[b-A-1][b-A-1] \
                                                 - f_h_h[i-1][i-1] - f_h_h[j-1][j-1]))
                if t_vec[a-A-1][b-A-1][i-1][j-1] != 0:
                    print('a=', a, ' ', 'b=',b, ' ', 'i=',i,' ','j=',j,\
                          ' ','t_vec=', t_vec[a-A-1][b-A-1][i-1][j-1])
                    

                    
for nn in range(iterations):
    chi_b_c = np.zeros((nocc_len,nocc_len))
    for b in nocc:
        for c in nocc:
            val = 0.0
            for d in nocc:
                for k in occ:
                    for l in occ:
                        val += (V_hh_pp[k-1][l-1][c-A-1][d-A-1]\
                            *t_vec[d-A-1][b-A-1][k-1][l-1])
            chi_b_c[b-A-1][c-A-1] =  val 
            
        
    chi_k_j = np.zeros((occ_len,occ_len)) 
    for k in occ:
        for j in occ:
            val = 0.0
            for d in nocc:
                for c in nocc:
                    for l in occ:
                        val+= (t_vec[c-A-1][d-A-1][l-1][j-1]*\
                              V_hh_pp[k-1][l-1][c-A-1][d-A-1])
            chi_k_j[k-1][j-1] = val
        

        
        
    chi_bk_jc = np.zeros((nocc_len,occ_len,occ_len,nocc_len)) 
    for b in nocc:
        for k in occ:
            for j in occ:
                for c in nocc:
                    val = 0.0
                    for d in nocc:
                        for l in occ:
                            val += (t_vec[d-A-1][b-A-1][l-1][j-1]*\
                                   V_hh_pp[k-1][l-1][c-A-1][d-A-1])
                    chi_bk_jc[b-A-1][k-1][j-1][c-A-1] = val
                
                
    chi_ab_cd = np.zeros((nocc_len,nocc_len,nocc_len,nocc_len))
    for a in nocc:
        for b in nocc:
            for c in nocc:
                for d in nocc:
                    val = 0.0
                    for k in occ:
                        for l in occ:
                            val += (t_vec[a-A-1][b-A-1][k-1][l-1]\
                                    *V_hh_pp[k-1][l-1][c-A-1][d-A-1])
                    chi_ab_cd[a-A-1][b-A-1][c-A-1][d-A-1] = val
                    
    
    

    
    #print("Setting up Hmat_ab_ij :")    
    Hmat_ab_ij = np.zeros((nocc_len,nocc_len,occ_len,occ_len))

    for a in nocc:
        for b in nocc:
            for i in occ:
                for j in occ:
                    Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1] = \
                    H_ab_ij(a,b,i,j,chi_b_c,chi_k_j,chi_bk_jc,chi_ab_cd,t_vec)\
                    /(f_p_p[a-A-1][a-A-1] + f_p_p[b-A-1][b-A-1] \
                      - f_h_h[i-1][i-1] - f_h_h[j-1][j-1])
#                     if Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1] != 0.0:
#                         print('a=', a, ' ', 'b=',b, ' ', 'i=',i,' ','j=',j, 'ME=',\
#                           ' ',Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1])
    
    t0 = t_vec - Hmat_ab_ij
    E_before = ground_E(t_vec)
    print(E_before)
    t_vec = t0
    print('t_ab_ij = ')
    #print(nn,t_vec)
    
    for a in nocc:
        for b in nocc:
            for i in occ:
                for j in occ:
                    if t0[a-A-1][b-A-1][i-1][j-1] != 0:
                        print('nn = ',nn,' ','t_nn=', t0[a-A-1][b-A-1][i-1][j-1])

                        
                        
    #calculation of ground state correlation energy w.r.t reference E_c
    
    change_E = ground_E(t0) - E_before
    print(change_E)
    print('nn', nn, ' ','ground state = ', ground_E(t0))
    if abs(change_E) <= 1e-10:
        break
    else:
        continue
    

#     if t_vec.all() == t0.all() :
#         break

                    
    
    
    
    

a= 5   b= 6   i= 1   j= 2   t_vec= -0.1
a= 5   b= 6   i= 2   j= 1   t_vec= 0.1
a= 5   b= 6   i= 3   j= 4   t_vec= -0.166666666667
a= 5   b= 6   i= 4   j= 3   t_vec= 0.166666666667
a= 6   b= 5   i= 1   j= 2   t_vec= 0.1
a= 6   b= 5   i= 2   j= 1   t_vec= -0.1
a= 6   b= 5   i= 3   j= 4   t_vec= 0.166666666667
a= 6   b= 5   i= 4   j= 3   t_vec= -0.166666666667
a= 7   b= 8   i= 1   j= 2   t_vec= -0.0714285714286
a= 7   b= 8   i= 2   j= 1   t_vec= 0.0714285714286
a= 7   b= 8   i= 3   j= 4   t_vec= -0.1
a= 7   b= 8   i= 4   j= 3   t_vec= 0.1
a= 8   b= 7   i= 1   j= 2   t_vec= 0.0714285714286
a= 8   b= 7   i= 2   j= 1   t_vec= -0.0714285714286
a= 8   b= 7   i= 3   j= 4   t_vec= 0.1
a= 8   b= 7   i= 4   j= 3   t_vec= -0.1
1.21904761905
t_ab_ij = 
nn =  0   t_nn= 0.054
nn =  0   t_nn= -0.054
nn =  0   t_nn= 0.0692592592593
nn =  0   t_nn= -0.0692592592593
nn =  0   t_nn= -0.054
nn =  0   t_nn= 0.054
nn =  0   t_nn= -0.0692592592593
nn =  0   t_nn= 0.0692592592593
nn =  0   t_nn= 0.0462682215743

In [90]:
#now we initialize t_vec using second perturbation theory

                
        
Hmat_ab_ij = np.zeros((nocc_len,nocc_len,occ_len,occ_len))

for a in nocc:
    for b in nocc:
        for i in occ:
            for j in occ:
                Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1] = H_ab_ij(a,b,i,j,t_vec)
                if Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1] != 0.0:
                    print('a=', a, ' ', 'b=',b, ' ', 'i=',i,' ','j=',j, 'ME=',\
                      ' ',Hmat_ab_ij[a-A-1][b-A-1][i-1][j-1])




a= 5   b= 6   i= 1   j= 2   t_vec= -0.0555555555556
a= 5   b= 6   i= 2   j= 1   t_vec= 0.0555555555556
a= 5   b= 6   i= 3   j= 4   t_vec= -0.1
a= 5   b= 6   i= 4   j= 3   t_vec= 0.1
a= 6   b= 5   i= 1   j= 2   t_vec= 0.0555555555556
a= 6   b= 5   i= 2   j= 1   t_vec= -0.0555555555556
a= 6   b= 5   i= 3   j= 4   t_vec= 0.1
a= 6   b= 5   i= 4   j= 3   t_vec= -0.1
a= 7   b= 8   i= 1   j= 2   t_vec= -0.0384615384615
a= 7   b= 8   i= 2   j= 1   t_vec= 0.0384615384615
a= 7   b= 8   i= 3   j= 4   t_vec= -0.0555555555556
a= 7   b= 8   i= 4   j= 3   t_vec= 0.0555555555556
a= 8   b= 7   i= 1   j= 2   t_vec= 0.0384615384615
a= 8   b= 7   i= 2   j= 1   t_vec= -0.0384615384615
a= 8   b= 7   i= 3   j= 4   t_vec= 0.0555555555556
a= 8   b= 7   i= 4   j= 3   t_vec= -0.0555555555556
a= 5   b= 6   i= 1   j= 2 ME=   -0.437606837607
a= 5   b= 6   i= 2   j= 1 ME=   0.437606837607
a= 5   b= 6   i= 3   j= 4 ME=   -0.422222222222
a= 5   b= 6   i= 4   j= 3 ME=   0.422222222222
a= 6   b= 5   i= 1   j= 2 ME=   0.

In [56]:
1.5-0.08336

1.41664

In [62]:
1.0-0.36956

0.63044